In [1]:
import pandas as pd
import csv
import numpy as np
from random import shuffle
import funcs as fc
import decimal

In [2]:
ctx = decimal.Context()
ctx.prec = 20

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

def getTwoNumbersGap(num1, num2):
    return abs(num1 - num2) + 1

def getRangeGap(data):
    maxV = max(data)
    minV = min(data)
    return getTwoNumbersGap(maxV, minV)

def getMinMaxFromMultiD(sourceList):
    maxValue = max(sourceList.reshape(-1))
    minValue = min(sourceList.reshape(-1))
    
    return maxValue, minValue

def getCategoryDiversityCNT(sourceList):
    return len(set([j for sub in sourceList for j in sub]))

In [3]:
def getOrgAllColumnDivertyCNT(datapath):
    org_moned_dataset = fc.readCSV(datapath, sep='\t')
    print(org_moned_dataset[0])
    org_moned_dataset = org_moned_dataset[1:]
    
    for dataRow in org_moned_dataset:
        dataRow[0] = int(dataRow[0])
        # 원본데이터 각 컬럼별 데이터 diversity count 추출
    
    columnsDiversityCNT = dict()
    
    for i, colName in enumerate(evalColNames):
        if colName in categoricalColNames:
            colIDX = evalColNames.index(colName)
            columnsDiversityCNT[colIDX] = len(set([data[i] for data in org_moned_dataset]))
        else:
            diversityCNT = getRangeGap([float(data[i]) for data in org_moned_dataset])
            columnsDiversityCNT[i] =diversityCNT       
        
        
    return columnsDiversityCNT
    

def getCatetoryColDiversityCNT(cellData):
    cnt = len(cellData.split(','))
    return cnt

def getNumericColDiversityCNT(cellData):
    numericData = [float(data.strip()) for data in cellData.split('-') if data.strip()]
    return getRangeGap(numericData)  

In [4]:
def getLEScore(normalizedDataSet): 
        
    eachColumnLEScoreList = list()    
    normalizedDataSet_cols =normalizedDataSet[0]
    print(normalizedDataSet_cols)
    totalColNum = len(normalizedDataSet_cols)
    
    for i, colName in enumerate(normalizedDataSet_cols):
        if colName not in evalColNames: continue
            
        colDataSet = [data[i] for data in normalizedDataSet[1:]]
        isCategoriCol = colName in categoricalColNames
        colDiversityCNT  = evalColDirvertyDIC[evalColNames.index(colName)]   
        
        colLEScoreList = list()  
        for i, cellData in enumerate(colDataSet):
            if isCategoriCol:
                cellDiversityCNT = getCatetoryColDiversityCNT(cellData)
                
            else:
                try:
                    cellDiversityCNT = getNumericColDiversityCNT(cellData)
                except:
                    print(i, cellData)
                    print(colName)
                    break
                
            
            
            colLEScoreList.append(abs(cellDiversityCNT-1)/abs(colDiversityCNT-1))
            
        colLEScore = sum(colLEScoreList) / len(colDataSet)
        print(colName ,"sum :", sum(colLEScoreList))
        print(colName ,"len :", len(colDataSet))
        print(colName ,"colLEScore :", colLEScore)
        print('')
        eachColumnLEScoreList.append(colLEScore)
        
    leScore = sum(eachColumnLEScoreList)/totalColNum 
    leScore = round(leScore, 7)
    print("totla LE SCORE :", leScore)
    
    return leScore

#### CONST VARs

In [5]:
evalColNames = ['AGE', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITION']
categoricalColNames = ['RACE',  'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITION']
originalDataSet_Path = './data/finalPatientDataSet_20.csv'

In [6]:
origin_dataset = fc.readCSV(originalDataSet_Path, sep='\t')

In [19]:
print(len(origin_dataset))

224886


In [8]:
len(set([data[-2] for data in origin_dataset]))

134

In [9]:
deathData = [data for data in origin_dataset if data[-1] == '1']
len(deathData)

29740

In [10]:
freqDict = dict()
for data in deathData:
    if data[-2] not in freqDict:
        freqDict[data[-2]] = 0
    freqDict[data[-2]] += 1
    
sorted(freqDict.items(), key=lambda x:x[1], reverse = True)

#### Origin Code Test

In [10]:
evalColDirvertyDIC = getOrgAllColumnDivertyCNT(originalDataSet_Path)
evalColDirvertyDIC

['AGE', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITIOIN', 'DEATH']


{0: 111.0, 1: 6, 2: 24, 3: 2, 4: 27, 5: 133}

In [15]:
org_moned_dataset = fc.readCSV('./normalized/recover_origin_wc.csv', sep='\t')
print(len(org_moned_dataset))

206290


In [12]:
org_moned_dataset[0]

['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']

In [13]:
leScore = getLEScore(org_moned_dataset)
float_to_str(leScore)

['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 1796.018181816946
AGE len : 206289
AGE colLEScore : 0.008706320656055078

BIRTHPLACE sum : 107.49999999999575
BIRTHPLACE len : 206289
BIRTHPLACE colLEScore : 0.0005211135833708814

CONDITION sum : 1072.9242424234246
CONDITION len : 206289
CONDITION colLEScore : 0.005201073457253778

ETHNICITY sum : 81.99999999999751
ETHNICITY len : 206289
ETHNICITY colLEScore : 0.00039750059382709455

GENDER sum : 0.0
GENDER len : 206289
GENDER colLEScore : 0.0

RACE sum : 8.200000000000003
RACE len : 206289
RACE colLEScore : 3.975005938271068e-05

totla LE SCORE : 0.0021237


'0.0021237'

In [17]:
berted_dataset_bert = fc.readCSV('./normalized/recover_bert_wc.csv', sep='\t')
print(len(berted_dataset_bert))

208298


In [18]:
leScor2 = getLEScore(berted_dataset_bert)
float_to_str(leScor2)

['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 0.0
AGE len : 208297
AGE colLEScore : 0.0

BIRTHPLACE sum : 3375.0384615351322
BIRTHPLACE len : 208297
BIRTHPLACE colLEScore : 0.01620301042038595

CONDITION sum : 5160.257575773075
CONDITION len : 208297
CONDITION colLEScore : 0.02477355687202924

ETHNICITY sum : 7166.39130435911
ETHNICITY len : 208297
ETHNICITY colLEScore : 0.03440467843684311

GENDER sum : 1274.0
GENDER len : 208297
GENDER colLEScore : 0.0061162666769084524

RACE sum : 6351.999999996525
RACE len : 208297
RACE colLEScore : 0.030494918313737233

totla LE SCORE : 0.0159989


'0.0159989'

#### Test Code